In [1]:
import pandas as pd

path="/kaggle/input/playground-series-s4e10/train.csv"
df=pd.read_csv(path)
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [2]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()

# Select the columns to be one-hot encoded
columns_to_encode = ['person_home_ownership','loan_intent', 'loan_grade', 'cb_person_default_on_file']

# Apply OneHotEncoder to those columns
encoded_data = ohe.fit_transform(df[columns_to_encode]).toarray()

# Convert the result to a DataFrame with proper column names
encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out(columns_to_encode))

# Concatenate the encoded columns back into the original DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Drop the original categorical columns
df = df.drop(columns_to_encode, axis=1)

print(df.head())

   id  person_age  person_income  person_emp_length  loan_amnt  loan_int_rate  \
0   0          37          35000                0.0       6000          11.49   
1   1          22          56000                6.0       4000          13.35   
2   2          29          28800                8.0       6000           8.90   
3   3          30          70000               14.0      12000          11.11   
4   4          22          60000                2.0       6000           6.92   

   loan_percent_income  cb_person_cred_hist_length  loan_status  \
0                 0.17                          14            0   
1                 0.07                           2            0   
2                 0.21                          10            0   
3                 0.17                           5            0   
4                 0.10                           3            0   

   person_home_ownership_MORTGAGE  ...  loan_intent_VENTURE  loan_grade_A  \
0                             0.0

In [3]:
y = df.iloc[:, 8]  # or df['loan_status']
X = df.drop(columns=['loan_status'])
feature_column_names = X.columns.tolist()
print("Feature column names (X):")
print(feature_column_names)
print("\nTarget (y):")
print(y.head())

Feature column names (X):
['id', 'person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length', 'person_home_ownership_MORTGAGE', 'person_home_ownership_OTHER', 'person_home_ownership_OWN', 'person_home_ownership_RENT', 'loan_intent_DEBTCONSOLIDATION', 'loan_intent_EDUCATION', 'loan_intent_HOMEIMPROVEMENT', 'loan_intent_MEDICAL', 'loan_intent_PERSONAL', 'loan_intent_VENTURE', 'loan_grade_A', 'loan_grade_B', 'loan_grade_C', 'loan_grade_D', 'loan_grade_E', 'loan_grade_F', 'loan_grade_G', 'cb_person_default_on_file_N', 'cb_person_default_on_file_Y']

Target (y):
0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64


In [4]:
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, VotingClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

In [5]:
model1 = RandomForestClassifier(n_estimators=100, random_state=42)
model2 = AdaBoostClassifier(n_estimators=100, random_state=42)
model3 = LGBMClassifier(n_estimators=100, random_state=42)
model4 = CatBoostClassifier(iterations=1000, silent=True, random_state=42)
model5 = ExtraTreesClassifier(n_estimators=100, random_state=42)
model6 = GradientBoostingClassifier(n_estimators=100, random_state=42)
model7 = XGBClassifier(n_estimators=100, use_label_encoder=False, eval_metric='logloss', random_state=42)


In [6]:
ensemble_model = VotingClassifier(
    estimators=[
        ('rf', model1), ('ada', model2), ('lgbm', model3), 
        ('catboost', model4), ('extra', model5), 
        ('gb', model6), ('xgb', model7)
    ],
    voting='soft'
)
ensemble_model.fit(X, y)

[LightGBM] [Info] Number of positive: 8350, number of negative: 50295
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012147 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1117
[LightGBM] [Info] Number of data points in the train set: 58645, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142382 -> initscore=-1.795644
[LightGBM] [Info] Start training from score -1.795644


VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('ada',
                              AdaBoostClassifier(n_estimators=100,
                                                 random_state=42)),
                             ('lgbm', LGBMClassifier(random_state=42)),
                             ('catboost',
                              <catboost.core.CatBoostClassifier object at 0x7fff713b9fc0>),
                             ('extra', ExtraTreesClassifier(random_state=42)),
                             ('gb',
                              GradientBoostingClassifier(random_state=42)),
                             ('x...
                                            grow_policy=None,
                                            importance_type=None,
                                            interaction_constraints=None,
                                            learning_rate=None, max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=None,
                                            max_leaves=None,
                                            min_child_weight=None, missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=100, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...))],
                 voting='soft')

In [7]:
test_path="/kaggle/input/playground-series-s4e10/test.csv"
df_test=pd.read_csv(test_path)
df_test.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,58645,23,69000,RENT,3.0,HOMEIMPROVEMENT,F,25000,15.76,0.36,N,2
1,58646,26,96000,MORTGAGE,6.0,PERSONAL,C,10000,12.68,0.10,Y,4
2,58647,26,30000,RENT,5.0,VENTURE,E,4000,17.19,0.13,Y,2
3,58648,33,50000,RENT,4.0,DEBTCONSOLIDATION,A,7000,8.90,0.14,N,7
4,58649,26,102000,MORTGAGE,8.0,HOMEIMPROVEMENT,D,15000,16.32,0.15,Y,4


In [8]:
# Apply OneHotEncoder to those columns
encoded_test_data = ohe.fit_transform(df_test[columns_to_encode]).toarray()

# Convert the result to a DataFrame with proper column names
encoded_df_test = pd.DataFrame(encoded_test_data, columns=ohe.get_feature_names_out(columns_to_encode))

# Concatenate the encoded columns back into the original DataFrame
df_test = pd.concat([df_test, encoded_df_test], axis=1)

# Drop the original categorical columns
df_test = df_test.drop(columns_to_encode, axis=1)

print(df_test.head())

      id  person_age  person_income  person_emp_length  loan_amnt  \
0  58645          23          69000                3.0      25000   
1  58646          26          96000                6.0      10000   
2  58647          26          30000                5.0       4000   
3  58648          33          50000                4.0       7000   
4  58649          26         102000                8.0      15000   

   loan_int_rate  loan_percent_income  cb_person_cred_hist_length  \
0          15.76                 0.36                           2   
1          12.68                 0.10                           4   
2          17.19                 0.13                           2   
3           8.90                 0.14                           7   
4          16.32                 0.15                           4   

   person_home_ownership_MORTGAGE  person_home_ownership_OTHER  ...  \
0                             0.0                          0.0  ...   
1                           

In [9]:
y_pred_proba = ensemble_model.predict_proba(df_test)[:, 1] 
threshold = 0.25 
y_pred = (y_pred_proba > threshold).astype(int)

In [10]:
results_df = pd.DataFrame({
    'id': df_test['id'],
    'prediction': y_pred
})

# Display the resulting DataFrame
print(results_df)

          id  prediction
0      58645           1
1      58646           0
2      58647           1
3      58648           0
4      58649           0
...      ...         ...
39093  97738           0
39094  97739           0
39095  97740           0
39096  97741           1
39097  97742           1

[39098 rows x 2 columns]


In [11]:
results_df.to_csv('/kaggle/output/submission.csv', index=False)